In [61]:
import pandas as pd
import os
import numpy as np

In [62]:
POSITION_BANK = ["President", "Chancellor", "Provost", "Director", "Dean", "Controller", "Trustee", "Member", "Regent", "Chairman", "Overseer", "Assistant", "Librarian", "Secretary", "Chaplain", "Minister", "Treasurer", "Senior Counsel", "General Counsel", "Legal Counsel", "University Counsel", "College Counsel", "Special Counsel", "Corporation Counsel", "Corporate Counsel", "Officer", "Chief", "Professor", "Commissioner", "Fellow", "Chairperson", "Manager", "Clergy", "Coordinator", "Auditor", "Governor", "Representative", "Stockbroker", "Advisor", "Commandant", "Rector", "Attorney", "Curator", "Clerk", "Department Head", "Pastor", "Head", "Comptroller", "Deputy", "Inspector General", "Instructor", "Registrar", "Ombuds", "Administrator", "Liaison", "Administrative Associate", "Webmaster", "Specialist", "University Planner", "Architect"]

path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "1999"

path_read = f"{path_connected_data}{year}_gptDataframe.csv"
split_df_path = f"{path_connected_data}{year}_split_positions.csv"
incorrect_path = f"{path_connected_data}{year}_incorrect.csv"

In [63]:
def parse_columns(path, size):
    df = pd.read_csv(path)
    all_columns = df.iloc[:, :size]
    return all_columns

In [64]:
#calculate number of key in a string (comma or and)
def calculate_occurrences(string, key):
    split_string = string.split(" ")
    and_count = 0
    if key == "," or key == ";":
        for word in split_string:
            if key in word:
                and_count+=1
    else:
        for word in split_string:
            if word == key:
                and_count+=1
    return and_count

def all_multiples(df):
    newdf = []
    for index, row in df.iterrows():
        position = row["Position"]        
        appears_twice =  check_appears_twice(position)
        if appears_twice:
            newdf.append(row)
    return pd.DataFrame(newdf)

def count_appearances(position):
    count = 0
    for word in POSITION_BANK:
        count += position.count(word)
    return count

def check_appears_twice(position):
    count = count_appearances(position)
    if count >= 2:
        return True
    return False

def extract_position(position):
    position = position.split(" ")
    for word in position:
        for key in POSITION_BANK:
            if key in word:
                return word


def split_multiple_positions(df):
    new_df = []
    for index, row in df.iterrows():
        position = row["Position"].title()
        name = row['Name']
        #if 'and' appears in the position column, and 2 or more elements from the word bank appear, then spit those rows
        and_appears = 'And' in position
        multiple_positions =  check_appears_twice(position)
        num_of_and = calculate_occurrences(position, "And")

        if and_appears and multiple_positions:
            #take substring containing first appearing position and substring with last appearing position
            #check that the substring following the last "and" actually contains a position, if not, go back one 'and'
            last_appearing_and = position.rfind('And')
            first_position = position[:last_appearing_and].strip()
            second_position = position[last_appearing_and + 3:].strip()

            #if there's a matching position on the right side of the last "and", split the string
            position_both_sides = sum(word in second_position for word in POSITION_BANK) >= 1
            if position_both_sides:
                pass
            else:
                #currently using "assistant" as a title, but this appears as extensions of titles, handled here
                #just add the entire position as 1
                if num_of_and == 1:
                    new_df.append(row)
                    # print(row)
                    continue
                #There is more than one "and", use the other "and" as the split location
                elif num_of_and >= 2:
                    last_appearing_and = first_position.rfind('And')
                    second_position = position[last_appearing_and + 3:].strip()
                    first_position = position[:last_appearing_and].strip()
                    
                position_both_sides = sum(word in second_position for word in POSITION_BANK) >= 1
                if not position_both_sides:
                    last_appearing_and = first_position.rfind('And')
                    second_position = position[last_appearing_and + 3:].strip()
                    first_position = position[:last_appearing_and].strip()
            position_both_sides = sum(word in second_position for word in POSITION_BANK) >= 1
            if position_both_sides:
                #append the positions to the new
                new_row = row.copy()
                #separate the rows into two
                row["Position"] = first_position
                new_row['Position'] = second_position.strip()
                new_df.append(row)
                new_df.append(new_row)
            else:
                new_df.append(row)
        else:
            #There is an "and" but only one position from the position bank appears or there is no "and"
            new_df.append(row)
    return pd.DataFrame(new_df)


def split_triple_positions(df, delimiter):
    #Split the people with 3 or more titles (rare)
    new_df = []
    for index, row in df.iterrows():
        position = row["Position"].title()
        #If 'and' appears in the position column, and 2 or more elements from the word bank appear, then split those rows
        appears_twice = check_appears_twice(position)
        num_of_delimiter = calculate_occurrences(position, delimiter)
        if num_of_delimiter > 0 and appears_twice:
            last_appearing_delimiter = position.rfind(delimiter)
            #Take substring containing first appearing position and substring with last appearing position
            first_position = position[:last_appearing_delimiter].strip()
            #Check that the substring following the last "and" actually contains a position, if not, go back one 'and'
            second_position = position[last_appearing_delimiter + 1:].strip()
            position_both_sides = sum(word in second_position for word in POSITION_BANK) >= 1

            if not position_both_sides:
                #Just add the entire position as 1
                if num_of_delimiter == 1:
                    new_df.append(row)
                    continue

                #Check where the second detected position is, with respect to location of commas
                last_appearing_delimiter = first_position.rfind(delimiter)
                second_position = position[last_appearing_delimiter + 1:].strip()
                first_position = position[:last_appearing_delimiter].strip()
                position_both_sides = sum(word in second_position for word in POSITION_BANK) >= 1
                if "Affleck" in row["Name"]:
                    print(first_position, "    ", second_position)
                    print(sum(word in second_position for word in POSITION_BANK))
                    print(position_both_sides)
                #two occurrences of the delimiter, but both positions that appear in the string are before the delimiter (append as one position)
                if not position_both_sides and num_of_delimiter == 2:
                    new_df.append(row)
                    continue
                elif position_both_sides and num_of_delimiter == 2:
                    new_row = row.copy()
                    row["Position"] = first_position
                    new_row['Position'] = second_position.strip()
                    new_df.append(row)
                    new_df.append(new_row)
                    continue
                else:
                    last_appearing_delimiter = first_position.rfind(delimiter)
                    second_position = position[last_appearing_delimiter + 1:].strip()
                    first_position = position[:last_appearing_delimiter].strip()
                    position_both_sides = sum(word in second_position for word in POSITION_BANK) >= 1
                    if not position_both_sides:
                        new_df.append(row)
                        continue
            #Position was found on both sides or too many delimiters
            new_row = row.copy()
            row["Position"] = first_position
            new_row['Position'] = second_position.strip()
            new_df.append(row)
            new_df.append(new_row)
        else:
            new_df.append(row)
    return pd.DataFrame(new_df)


def merge_incorrectly_split(df, delimiter):
    df = df.reset_index(drop=True) 
    index = 1
    while index < len(df):
        row = df.loc[index]
        position = row["Position"].lower().title()
        name = row["Name"].lower().title()
        num_commas = calculate_occurrences(position, delimiter)
        #No commas and position contains "Office Of", merge two positions
        if num_commas == 0 and "Office Of" in position:
            if index - 1 < len(df): 
                prev_name = df.at[index - 1, "Name"]
                if name == prev_name:
                    print(df.at[index,"Position"])
                    df.at[index, "Position"] = df.at[index + 1, "Position"] + ", " + df.at[index, "Position"]
                    df = df.drop(index - 1).reset_index(drop=True) 
        #only position is a plural with ' (position = President's Office)
        num_positions = count_appearances(position)
        if num_positions == 1:
            spec_position = extract_position(position)
            if "'s" in str(spec_position).lower() and "office" in position.lower():
                prev_name = df.at[index - 1, "Name"]
                if name == prev_name:
                    print(df.at[index,"Position"])
                    df.at[index, "Position"] = df.at[index - 1, "Position"] + ", " + df.at[index, "Position"]
                    df = df.drop(index - 1).reset_index(drop=True)
        index += 1  
    return pd.DataFrame(df)
        

In [65]:
full_dataframe = parse_columns(path_read,8)

In [66]:
#call each method more than once to account for 4 position cells
split_dataframe_all = split_triple_positions(full_dataframe, ";")
split_dataframe_all = split_triple_positions(split_dataframe_all, ";")
split_dataframe_all = split_multiple_positions(split_dataframe_all)
split_dataframe_all = split_multiple_positions(split_dataframe_all)
split_dataframe_all = split_triple_positions(split_dataframe_all, ",")
split_dataframe_all = split_triple_positions(split_dataframe_all, ",")
split_dataframe_all = merge_incorrectly_split(split_dataframe_all, ",")
# split_dataframe_all = merge_incorrectly_split(split_dataframe_all, ";")

Controller'S Office
Office Of The Provost


In [67]:
split_dataframe_all.to_csv(split_df_path, index = False)

In [68]:
def double_positions(df):
    for index, row in df.iterrows():
        position = row["Position"].title()
        num_positions = count_appearances(position)
        if num_positions >= 2:
            print(position)

def semicolon_positions(df):
    for index, row in df.iterrows():
        position = row["Position"].title()
        if ";" in position:
            print(position)

double_positions(split_dataframe_all)

Assistant Secretary
Assistant Secretary
Assistant Secretary
Deputy To The President
Assistant Secretary To The Board Of Trustees
Special Assistant To The President
Chief Administrative Officer
Assistant Vice President, Development
Assistant To The President
Assistant Secretary
Special Assistant To The President
Administrative Assistant To The Dean
Assistant Vice President, Student Services
Assistant Vice President, Administrative Services
Assistant Treasurer
Assistant Secretary
Assistant To The Provost
Assistant To The Executive Vice President
Assistant To The President
Assistant To The President
Assistant Vice President, Academic Affairs
Assistant Vice President, Academic Affairs
Assistant Vice President, Enrollment Management
Assistant Vice President, Finance
Assistant Secretary
Assistant Secretary
Assistant To The President
Executive Assistant President
Assistant Vice President
Assistant Vice President
Executive Assistant To The President
Assistant Vice President, Student Affairs
As

In [69]:
semicolon_positions(full_dataframe)